In [1]:
!pip install transformers==3.0.2

     |████████████████████████████████| 769 kB 1.4 MB/s 
     |████████████████████████████████| 3.0 MB 5.8 MB/s 
ERROR: allennlp 1.0.0 has requirement transformers<2.12,>=2.9, but you'll have transformers 3.0.2 which is incompatible.
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.7.0
    Uninstalling tokenizers-0.7.0:
      Successfully uninstalled tokenizers-0.7.0
  Attempting uninstall: transformers
    Found existing installation: transformers 2.11.0
    Uninstalling transformers-2.11.0:
      Successfully uninstalled transformers-2.11.0
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import gc
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/janatahack-independence-day-2020-ml-hackathon/test.csv
/kaggle/input/janatahack-independence-day-2020-ml-hackathon/train.csv
/kaggle/input/janatahack-independence-day-2020-ml-hackathon/sample_submission_UVKGLZE.csv
/kaggle/input/janatahack/test.csv
/kaggle/input/janatahack/sample_submission.csv
/kaggle/input/janatahack/train.csv


In [3]:
train = pd.read_csv("/kaggle/input/janatahack-independence-day-2020-ml-hackathon/train.csv")
train.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [4]:
test = pd.read_csv("/kaggle/input/janatahack-independence-day-2020-ml-hackathon/test.csv")
test.head()

,ID,TITLE,ABSTRACT
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...


In [5]:
sample = pd.read_csv("/kaggle/input/janatahack-independence-day-2020-ml-hackathon/sample_submission_UVKGLZE.csv")
sample.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,0,0,0
1,20974,0,0,0,0,0,0
2,20975,0,0,0,0,0,0
3,20976,0,0,0,0,0,0
4,20977,0,0,0,0,0,0


In [6]:
list_classes = ["Computer Science", "Physics", "Mathematics", "Statistics", "Quantitative Biology", "Quantitative Finance"]
y = train[list_classes].values

In [7]:
y

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0],
       [0, 0, 1, 1, 0, 0]])

In [8]:
import tensorflow as tf
import transformers
from sklearn.model_selection import KFold

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [9]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

In [10]:
#!pip install transformers

In [11]:
transformers.__version__

'3.0.2'

In [12]:
from transformers import AutoTokenizer, TFAutoModel

In [13]:
epochs = 4
maxlen = 150

model_name = "roberta-base"

batch_size = 8 * strategy.num_replicas_in_sync

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [15]:
train.TITLE.values

array(['Reconstructing Subject-Specific Effect Maps',
       'Rotation Invariance Neural Network',
       'Spherical polyharmonics and Poisson kernels for polyharmonic functions',
       ..., 'Analysing Soccer Games with Clustering and Conceptors',
       'On the Efficient Simulation of the Left-Tail of the Sum of Correlated Log-normal Variates',
       'Why optional stopping is a problem for Bayesians'], dtype=object)

In [16]:
%%time
train_encode = tokenizer(list(train.TITLE.values), list(train.ABSTRACT.values), 
                      max_length=maxlen, return_tensors="np", padding=True, 
                      return_token_type_ids=True, return_attention_mask=True)

CPU times: user 1min 1s, sys: 1.24 s, total: 1min 2s
Wall time: 1min 2s


In [17]:
train_encode

{'input_ids': array([[    0, 21109,   261, ...,     1,     1,     1],
       [    0,   500, 35915, ...,     1,     1,     1],
       [    0, 12582, 48152, ...,     1,     1,     1],
       ...,
       [    0,  4688, 38711, ...,     1,     1,     1],
       [    0,  4148,     5, ...,     1,     1,     1],
       [    0,  7608, 17679, ...,     1,     1,     1]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}

In [18]:
def get_model(maxlen=150):
    
    #base_model = TFDistilBertModel.from_pretrained("distilbert-base-multilingual-cased")
    
    base_model = TFAutoModel.from_pretrained(model_name)
    
    input_ids = tf.keras.layers.Input(shape =(maxlen, ), dtype=tf.int32, name="input_ids")
    input_type = tf.keras.layers.Input(shape =(maxlen, ), dtype=tf.int32, name="token_type_ids")
    input_mask = tf.keras.layers.Input(shape =(maxlen, ), dtype=tf.int32, name="attention_mask")
    
    
    embedding = base_model([input_ids, input_mask, input_type])[0]
    #embedding = base_model([input_ids, input_mask])[0]
    
    print(embedding.shape)
    
    output = tf.keras.layers.Dense(6, activation="sigmoid")(embedding[:, 0, :])
    
    model = tf.keras.models.Model(inputs=[input_ids, input_mask, input_type], outputs = output)
    
    model.compile(tf.keras.optimizers.Adam(1e-5), "binary_crossentropy", ["accuracy"])
    
    return model

In [19]:
with strategy.scope():
    cls_model = get_model(maxlen)
    cls_model.summary()


(None, 150, 768)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 150)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode ((None, 150, 768), ( 124645632   input_ids[0][0]                  
                                                                 attention_m

In [20]:
%%time
ps = cls_model([train_encode['input_ids'][:10], train_encode['attention_mask'][:10], train_encode['token_type_ids'][:10]])

CPU times: user 639 ms, sys: 206 ms, total: 844 ms
Wall time: 5.18 s


In [21]:
fold = KFold(n_splits=4, shuffle=True, random_state=108)

In [22]:
%%time
hists = []
models = []
for i, (train_idx, val_idx) in enumerate(fold.split(np.arange(y.shape[0]))):
    print(f"----FOLD: {i+1}----\n",train_idx, val_idx)
    
    
    x_train = [train_encode['input_ids'][train_idx], 
               train_encode['attention_mask'][train_idx], 
               train_encode['token_type_ids'][train_idx]]
    
    y_train = y[train_idx]
    
    x_val = [train_encode['input_ids'][val_idx],
             train_encode['attention_mask'][val_idx],
             train_encode['token_type_ids'][val_idx]]
    y_val = y[val_idx]
    
    
    hist=cls_model.fit(x_train, y_train,
                       epochs=4, 
                       batch_size = batch_size,
                       validation_data = (x_val, y_val),
                      )
    hists.append(hist)
    #models.append(cls_model)
    
    gc.collect()

----FOLD: 1----
 [    0     3     4 ... 20967 20968 20971] [    1     2     9 ... 20964 20969 20970]
Epoch 1/4
246/246 [==============================] - 139s 565ms/step - accuracy: 0.6701 - loss: 0.2471 - val_accuracy: 0.7271 - val_loss: 0.1870
Epoch 2/4
246/246 [==============================] - 98s 399ms/step - accuracy: 0.7692 - loss: 0.1712 - val_accuracy: 0.7913 - val_loss: 0.1767
Epoch 3/4
246/246 [==============================] - 98s 399ms/step - accuracy: 0.7896 - loss: 0.1523 - val_accuracy: 0.7847 - val_loss: 0.1727
Epoch 4/4
246/246 [==============================] - 98s 398ms/step - accuracy: 0.8063 - loss: 0.1384 - val_accuracy: 0.7763 - val_loss: 0.1706
----FOLD: 2----
 [    0     1     2 ... 20967 20969 20970] [   12    19    27 ... 20965 20968 20971]
Epoch 1/4
246/246 [==============================] - 98s 398ms/step - accuracy: 0.8016 - loss: 0.1418 - val_accuracy: 0.8306 - val_loss: 0.1114
Epoch 2/4
246/246 [==============================] - 98s 398ms/step - accurac

In [23]:
gc.collect()

20

In [24]:
%%time
test_encode = tokenizer(list(test.TITLE.values), list(test.ABSTRACT.values), 
                      max_length=maxlen, return_tensors="tf", padding=True, 
                      return_token_type_ids=True, return_attention_mask=True)

CPU times: user 19.7 s, sys: 325 ms, total: 20 s
Wall time: 20 s


In [25]:
ps = cls_model.predict([test_encode['input_ids'], test_encode['attention_mask'], test_encode['token_type_ids']],
                      verbose=1, batch_size=batch_size)

141/141 [==============================] - 22s 158ms/step


In [26]:
ps

array([[8.57650042e-02, 1.43885612e-04, 1.03149116e-02, 9.99757528e-01,
        1.78843737e-04, 9.04291868e-04],
       [2.15768814e-05, 9.99910593e-01, 2.15172768e-05, 5.10036945e-04,
        1.77025795e-05, 8.65757465e-05],
       [9.99931753e-01, 2.57313251e-04, 7.95817375e-03, 7.21812248e-04,
        1.14530325e-04, 1.71869993e-04],
       ...,
       [8.85432959e-03, 1.14756823e-03, 5.90682030e-05, 2.40893751e-01,
        9.28167224e-01, 1.44892931e-03],
       [5.86247444e-03, 7.66545534e-04, 6.51940405e-02, 9.99529004e-01,
        1.20881200e-03, 2.20295787e-03],
       [8.82746339e-01, 1.92910433e-04, 9.55258191e-01, 7.34031200e-05,
        9.55462456e-05, 1.07288361e-04]], dtype=float32)

In [27]:
type(ps)

numpy.ndarray

In [28]:
ps1 = np.round(ps, 2)

In [29]:
ps1

array([[0.09, 0.  , 0.01, 1.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  ],
       [1.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       ...,
       [0.01, 0.  , 0.  , 0.24, 0.93, 0.  ],
       [0.01, 0.  , 0.07, 1.  , 0.  , 0.  ],
       [0.88, 0.  , 0.96, 0.  , 0.  , 0.  ]], dtype=float32)

In [30]:
ps2 = np.around(ps1)
ps2

array([[0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 1., 0., 0., 0.]], dtype=float32)

In [31]:
ps2.shape

(8989, 6)

In [32]:
ps2[:,0]

array([0., 0., 1., ..., 0., 0., 1.], dtype=float32)

In [33]:
sample['Computer Science']=ps2[:,0]
sample['Physics']=ps2[:,1]
sample['Mathematics']=ps2[:,2]
sample['Statistics']=ps2[:,3]
sample['Quantitative Biology']=ps2[:,4]
sample['Quantitative Finance']=ps2[:,5]

In [34]:
sample.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0.0,0.0,0.0,1.0,0.0,0.0
1,20974,0.0,1.0,0.0,0.0,0.0,0.0
2,20975,1.0,0.0,0.0,0.0,0.0,0.0
3,20976,0.0,1.0,0.0,0.0,0.0,0.0
4,20977,1.0,0.0,0.0,0.0,0.0,0.0


In [35]:
sample.to_csv('submission.csv', index=False)